In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import random
import pickle
import math
import time
import os
from utils import create_network_df, generate_od_matrices, prepare_network_data
from leblanc import leblanc_algorithm
from tqdm import tqdm

pwd = os.path.join(os.getenv('PWD'), 'traffic_assignment')
filepath = 'data/sioux'

In [3]:
sioux = create_network_df(network_name="SiouxFalls")
T_0, C = prepare_network_data(sioux)
eps = 0.005

In [5]:
D = np.array([
    [0, 100, 100, 500, 200, 300, 500, 800, 500, 1300, 500, 200, 500, 300, 500, 500, 400, 100, 300, 300, 100, 400, 300, 100],
    [100, 0, 100, 200, 100, 400, 200, 400, 200, 600, 200, 100, 300, 100, 100, 400, 200, 0, 100, 100, 0, 100, 0, 0],
    [100, 100, 0, 200, 100, 300, 100, 200, 100, 300, 300, 200, 100, 100, 100, 200, 100, 0, 0, 0, 0, 100, 100, 0],
    [500, 200, 200, 0, 500, 400, 400, 700, 700, 1200, 1400, 600, 600, 500, 500, 800, 500, 100, 200, 300, 200, 400, 500, 200],
    [200, 100, 100, 500, 0, 200, 200, 500, 800, 1000, 500, 200, 200, 100, 200, 500, 200, 0, 100, 100, 100, 200, 100, 0],
    [300, 400, 300, 400, 200, 0, 400, 800, 400, 800, 400, 200, 200, 100, 200, 900, 500, 100, 200, 300, 100, 200, 100, 100],
    [500, 200, 100, 400, 200, 400, 0, 1000, 600, 1900, 500, 700, 400, 200, 500, 1400, 1000, 200, 400, 500, 200, 500, 200, 100],
    [800, 400, 200, 700, 500, 800, 1000, 0, 800, 1600, 800, 600, 600, 400, 600, 2200, 1400, 300, 700, 900, 400, 500, 300, 200],
    [500, 200, 100, 700, 800, 400, 600, 800, 0, 2800, 1400, 600, 600, 600, 900, 1400, 900, 200, 400, 600, 300, 700, 500, 200],
    [1300, 600, 300, 1200, 1000, 800, 1900, 1600, 2800, 0, 4000, 2000, 1900, 2100, 4000, 4400, 3900, 700, 1800, 2500, 1200, 2600, 1800, 800],
    [500, 200, 300, 1500, 500, 400, 500, 800, 1400, 3900, 0, 1400, 1000, 1600, 1400, 1400, 1000, 100, 400, 600, 400, 1100, 1300, 600],
    [200, 100, 200, 600, 200, 200, 700, 600, 600, 2000, 1400, 0, 1300, 700, 700, 700, 600, 200, 300, 400, 300, 700, 700, 500],
    [500, 300, 100, 600, 200, 200, 400, 600, 600, 1900, 1000, 1300, 0, 600, 700, 600, 500, 100, 300, 600, 600, 1300, 800, 800],
    [300, 100, 100, 500, 100, 100, 200, 400, 600, 2100, 1600, 700, 600, 0, 1300, 700, 700, 100, 300, 500, 400, 1200, 1100, 400],
    [500, 100, 100, 500, 200, 200, 500, 600, 1000, 4000, 1400, 700, 700, 1300, 0, 1200, 1500, 200, 800, 1100, 800, 2600, 1000, 400],
    [500, 400, 200, 800, 500, 900, 1400, 2200, 1400, 4400, 1400, 700, 600, 700, 1200, 0, 2800, 500, 1300, 1600, 600, 1200, 500, 300],
    [400, 200, 100, 500, 200, 500, 1000, 1400, 900, 3900, 1000, 600, 500, 700, 1500, 2800, 0, 600, 1700, 1700, 600, 1700, 600, 300],
    [100, 0, 0, 100, 0, 100, 200, 300, 200, 700, 200, 200, 100, 100, 200, 500, 600, 0, 300, 400, 100, 300, 100, 0],
    [300, 100, 0, 200, 100, 200, 400, 700, 400, 1800, 400, 300, 300, 300, 800, 1300, 1700, 300, 0, 1200, 400, 1200, 300, 100],
    [300, 100, 0, 300, 100, 300, 500, 900, 600, 2500, 600, 500, 600, 500, 1100, 1600, 1700, 400, 1200, 0, 1200, 2400, 700, 400],
    [100, 0, 0, 200, 100, 100, 200, 400, 300, 1200, 400, 300, 600, 400, 800, 600, 600, 100, 400, 1200, 0, 1800, 700, 500],
    [400, 100, 100, 400, 200, 200, 500, 500, 700, 2600, 1100, 700, 1300, 1200, 2600, 1200, 1700, 300, 1200, 2400, 1800, 0, 2100, 1100],
    [300, 0, 100, 500, 100, 100, 200, 300, 500, 1800, 1300, 700, 800, 1100, 1000, 500, 600, 100, 300, 700, 700, 2100, 0, 700],
    [100, 0, 0, 200, 0, 100, 100, 200, 200, 800, 600, 500, 700, 400, 400, 300, 300, 0, 100, 400, 500, 1100, 700, 0]
])

In [6]:
os.makedirs(filepath + '/uncongested', exist_ok=True)
os.makedirs(filepath + '/moderate', exist_ok=True)
os.makedirs(filepath + '/congested', exist_ok=True)

In [7]:
num_matrices=5000
uncongested_matrices = generate_od_matrices(D, num_matrices, 'uncongested')
moderate_matrices = generate_od_matrices(D, num_matrices, 'moderate')
congested_matrices = generate_od_matrices(D, num_matrices, 'congested')

In [9]:
def create_index_file(condition, iters, times, eps, filepath='data/sioux'):
    files = os.listdir(filepath + f"/{condition}")
    indices = [int(f.split('_')[1].split('.')[0]) for f in files if f.startswith('sample_')]
    
    df = pd.DataFrame({
        'condition': condition,
        'eps' : eps,
        'sample_id': indices,
        'iterations' : iters,
        'execution_time' : times,
        'filename': [filepath + f"/{condition}/sample_{i:04d}.pkl" for i in indices]
    })
    df.to_csv(filepath + f"/{condition}_index.csv", index=False)

def save_dataset(condition='uncongested', num_samples=5000):
    start_time = time.time()
    od_matrices = generate_od_matrices(D, num_samples, condition)
    times = []
    iters = []
    for i in tqdm(range(num_samples), desc=f"Обработка {condition}"):
        result_matrix, iter_num = leblanc_algorithm(T_0, od_matrices[i], C, eps)
        end_time = time.time()
        total_time = end_time - start_time
        times.append(total_time)
        iters.append(iter_num)
        metadata = {
            'iterations': iter_num,
            'execution_time': total_time,
        }
        data_pair = {
            'input': od_matrices[i],
            'output': result_matrix,
            'metadata' : metadata
        }
        
        filename = filepath + f"/{condition}/sample_{i:04d}.pkl"
        with open(filename, 'wb') as f:
            pickle.dump(data_pair, f)

    create_index_file(condition, iters, times, eps)

In [31]:
save_dataset(num_samples=1000)

Обработка uncongested:   0%|          | 0/1000 [00:00<?, ?it/s]

Обработка uncongested: 100%|██████████| 1000/1000 [54:27<00:00,  3.27s/it] 


In [33]:
save_dataset(condition='moderate', num_samples=1000)

Обработка moderate:   0%|          | 0/1000 [00:00<?, ?it/s]/home/polina/kans/leblanc.py:27: RuntimeWarning: invalid value encountered in divide
  T = T_0 * (1 + 0.15 * (X / C)**4)
/home/polina/kans/leblanc.py:35: RuntimeWarning: divide by zero encountered in divide
  C_inv_4 = 1 / C**4
Обработка moderate: 100%|██████████| 1000/1000 [29:53<00:00,  1.79s/it]


In [ ]:
save_dataset(condition='congested', num_samples=1000)

Обработка congested:  49%|████▊     | 487/1000 [30:08<35:01,  4.10s/it] 